# Chapter 37: Linting and Formatting

Consistent code style and automated quality checks are essential for maintaining large TypeScript codebases. Linting catches potential bugs and enforces best practices, while formatting ensures visual consistency. This chapter covers setting up robust tooling pipelines for TypeScript projects.

---

## 37.1 ESLint for TypeScript

ESLint is the standard JavaScript linter, and `@typescript-eslint` extends it with TypeScript-specific rules that leverage the type system for deeper analysis.

### Installation and Basic Configuration

```bash
# Install ESLint and TypeScript plugins
npm install -D eslint @typescript-eslint/parser @typescript-eslint/eslint-plugin

# For React projects
npm install -D eslint-plugin-react eslint-plugin-react-hooks

# For Vue projects
npm install -D eslint-plugin-vue

# For Node.js
npm install -D eslint-plugin-node
```

**eslint.config.js (Flat Config - Modern Approach):**

```javascript
// eslint.config.js
import js from '@eslint/js';
import tsParser from '@typescript-eslint/parser';
import tsPlugin from '@typescript-eslint/eslint-plugin';
import globals from 'globals';

export default [
  // Base JavaScript rules
  js.configs.recommended,
  
  // TypeScript configuration
  {
    files: ['**/*.ts', '**/*.tsx'],
    languageOptions: {
      parser: tsParser,
      parserOptions: {
        project: './tsconfig.json',
        tsconfigRootDir: import.meta.dirname,
        ecmaVersion: 'latest',
        sourceType: 'module'
      },
      globals: {
        ...globals.browser,
        ...globals.node,
        ...globals.es2021
      }
    },
    plugins: {
      '@typescript-eslint': tsPlugin
    },
    rules: {
      ...tsPlugin.configs.recommended.rules,
      ...tsPlugin.configs['recommended-requiring-type-checking'].rules,
      
      // Override specific rules
      '@typescript-eslint/explicit-function-return-type': 'error',
      '@typescript-eslint/no-explicit-any': 'error',
      '@typescript-eslint/strict-boolean-expressions': 'error'
    }
  },
  
  // Ignore patterns
  {
    ignores: [
      'dist/**',
      'node_modules/**',
      '*.config.js',
      'coverage/**'
    ]
  }
];
```

**Legacy .eslintrc.json (if needed):**

```json
{
  "root": true,
  "parser": "@typescript-eslint/parser",
  "parserOptions": {
    "project": "./tsconfig.json",
    "tsconfigRootDir": ".",
    "ecmaVersion": 2022,
    "sourceType": "module"
  },
  "plugins": ["@typescript-eslint"],
  "extends": [
    "eslint:recommended",
    "plugin:@typescript-eslint/recommended",
    "plugin:@typescript-eslint/recommended-requiring-type-checking",
    "plugin:@typescript-eslint/strict-type-checked"
  ],
  "rules": {
    "@typescript-eslint/explicit-function-return-type": "error",
    "@typescript-eslint/no-unused-vars": ["error", { 
      "argsIgnorePattern": "^_",
      "varsIgnorePattern": "^_" 
    }],
    "@typescript-eslint/naming-convention": [
      "error",
      {
        "selector": "interface",
        "format": ["PascalCase"],
        "prefix": ["I"]
      },
      {
        "selector": "typeAlias",
        "format": ["PascalCase"]
      },
      {
        "selector": "variable",
        "format": ["camelCase", "UPPER_CASE"],
        "leadingUnderscore": "allow"
      }
    ]
  },
  "ignorePatterns": ["dist", "node_modules", "*.js"]
}
```

**Explanation:**
- `parser`: `@typescript-eslint/parser` allows ESLint to understand TypeScript syntax
- `project`: Points to your tsconfig.json, enabling rules that use type information
- `recommended-requiring-type-checking`: Rules that need the TypeScript type checker (slower but more powerful)
- `strict-type-checked`: Maximum strictness with type-aware rules

### Type-Aware Linting Rules

Rules that require type information catch more sophisticated issues:

```typescript
// .eslintrc.json rules section
{
  "rules": {
    // Requires types in catch clauses to be unknown or any
    "@typescript-eslint/use-unknown-in-catch-callback-variable": "error",
    
    // Prevents promises in places not designed to handle them
    "@typescript-eslint/no-floating-promises": "error",
    
    // Requires async functions to have await
    "@typescript-eslint/require-await": "error",
    
    // Prevents unsafe assignment of any values
    "@typescript-eslint/no-unsafe-assignment": "error",
    
    // Prevents unsafe member access on any values
    "@typescript-eslint/no-unsafe-member-access": "error",
    
    // Prevents unsafe return of any values
    "@typescript-eslint/no-unsafe-return": "error",
    
    // Enforces strict equality (=== and !==)
    "@typescript-eslint/strict-boolean-expressions": ["error", {
      "allowNullableBoolean": false,
      "allowNullableString": false,
      "allowNullableNumber": false
    }],
    
    // Requires explicit return types on functions
    "@typescript-eslint/explicit-function-return-type": ["error", {
      "allowExpressions": true,
      "allowTypedFunctionExpressions": true
    }],
    
    // Prevents misuse of Promises
    "@typescript-eslint/no-misused-promises": ["error", {
      "checksVoidReturn": {
        "arguments": true,
        "attributes": true
      }
    }],
    
    // Enforces proper handling of async functions
    "@typescript-eslint/no-unsafe-argument": "error",
    
    // Prevents calling values with type any
    "@typescript-eslint/no-unsafe-call": "error"
  }
}
```

**Example of type-aware rules in action:**

```typescript
// ❌ Caught by @typescript-eslint/no-floating-promises
async function fetchData() {
  const response = fetch('/api/data'); // Missing await
  return response.json(); // TypeError: response.json is not a function
}

// ✅ Fixed
async function fetchData() {
  const response = await fetch('/api/data');
  return response.json();
}

// ❌ Caught by @typescript-eslint/no-misused-promises
const button = document.getElementById('btn');
button?.addEventListener('click', async () => {
  await saveData(); // Async function in void-return position
});

// ✅ Fixed - handle the promise
button?.addEventListener('click', () => {
  saveData().catch(console.error);
});

// ❌ Caught by @typescript-eslint/strict-boolean-expressions
function checkUser(user: User | null) {
  if (user) { // OK - explicit null check
    return user.name;
  }
}

function checkString(value: string) {
  if (value) { // Error - truthy check on string
    return value.toUpperCase();
  }
}

// ✅ Fixed
function checkString(value: string) {
  if (value !== '') {
    return value.toUpperCase();
  }
}
```

---

## 37.2 Prettier Integration

Prettier handles formatting while ESLint handles code quality. They must be configured to work together.

### Installation and Configuration

```bash
npm install -D prettier eslint-config-prettier eslint-plugin-prettier
```

**.prettierrc.json:**

```json
{
  "semi": true,
  "trailingComma": "es5",
  "singleQuote": true,
  "printWidth": 100,
  "tabWidth": 2,
  "useTabs": false,
  "bracketSpacing": true,
  "arrowParens": "avoid",
  "endOfLine": "lf",
  "overrides": [
    {
      "files": "*.ts",
      "options": {
        "parser": "typescript"
      }
    },
    {
      "files": "*.json",
      "options": {
        "parser": "json",
        "tabWidth": 2
      }
    }
  ]
}
```

**.prettierignore:**

```
dist
node_modules
coverage
*.min.js
*.min.css
package-lock.json
yarn.lock
```

### ESLint + Prettier Integration

```javascript
// eslint.config.js
import js from '@eslint/js';
import tsParser from '@typescript-eslint/parser';
import tsPlugin from '@typescript-eslint/eslint-plugin';
import prettierPlugin from 'eslint-plugin-prettier';
import prettierConfig from 'eslint-config-prettier';

export default [
  js.configs.recommended,
  {
    files: ['**/*.ts', '**/*.tsx'],
    languageOptions: {
      parser: tsParser,
      parserOptions: {
        project: './tsconfig.json'
      }
    },
    plugins: {
      '@typescript-eslint': tsPlugin,
      'prettier': prettierPlugin
    },
    rules: {
      ...tsPlugin.configs.recommended.rules,
      
      // Prettier integration
      ...prettierConfig.rules,
      'prettier/prettier': 'error', // Treat formatting issues as errors
      
      // Disable rules that conflict with Prettier
      '@typescript-eslint/indent': 'off',
      'max-len': 'off',
      'semi': 'off',
      '@typescript-eslint/semi': 'off',
      'quotes': 'off',
      '@typescript-eslint/quotes': 'off'
    }
  }
];
```

**Explanation:**
- `eslint-config-prettier`: Disables ESLint rules that conflict with Prettier
- `eslint-plugin-prettier`: Runs Prettier as an ESLint rule
- `prettier/prettier: 'error'`: Makes formatting issues fail the build
- Conflicting rules (indent, quotes, semi) must be disabled in ESLint

### Running Prettier

```json
// package.json scripts
{
  "scripts": {
    "format": "prettier --write \"src/**/*.{ts,tsx,json,css,md}\"",
    "format:check": "prettier --check \"src/**/*.{ts,tsx,json,css,md}\"",
    "lint": "eslint . --ext .ts,.tsx",
    "lint:fix": "eslint . --ext .ts,.tsx --fix",
    "type-check": "tsc --noEmit"
  }
}
```

---

## 37.3 Import/Export Organization

Consistent import ordering and export patterns improve readability and prevent circular dependencies.

### ESLint Import Plugin

```bash
npm install -D eslint-plugin-import eslint-import-resolver-typescript
```

```javascript
// eslint.config.js
import importPlugin from 'eslint-plugin-import';

export default [
  // ...other configs
  {
    files: ['**/*.ts', '**/*.tsx'],
    plugins: {
      'import': importPlugin
    },
    settings: {
      'import/resolver': {
        typescript: {
          alwaysTryTypes: true,
          project: './tsconfig.json'
        }
      }
    },
    rules: {
      // Enforce imports come before other statements
      'import/first': 'error',
      
      // Prevent duplicate imports
      'import/no-duplicates': 'error',
      
      // Ensure imports point to resolvable files/modules
      'import/no-unresolved': 'error',
      
      // Prevent importing from devDependencies in production code
      'import/no-extraneous-dependencies': ['error', {
        'devDependencies': [
          '**/*.test.ts',
          '**/*.spec.ts',
          'vite.config.ts',
          'vitest.config.ts'
        ]
      }],
      
      // Enforce consistent import ordering
      'import/order': ['error', {
        'groups': [
          'builtin',      // Node.js built-ins (fs, path)
          'external',     // npm packages (react, lodash)
          'internal',     // Aliased imports (@/*)
          'parent',       // ../
          'sibling',      // ./
          'index',        // ./
          'object',       // import { type X }
          'type'          // import type {}
        ],
        'pathGroups': [
          {
            'pattern': '@/**',
            'group': 'internal',
            'position': 'before'
          }
        ],
        'pathGroupsExcludedImportTypes': ['builtin'],
        'newlines-between': 'always',
        'alphabetize': {
          'order': 'asc',
          'caseInsensitive': true
        }
      }],
      
      // Prefer type imports when possible
      '@typescript-eslint/consistent-type-imports': ['error', {
        'prefer': 'type-imports',
        'fixStyle': 'inline-type-imports'
      }],
      
      // No relative parent imports beyond one level
      'no-restricted-imports': ['error', {
        'patterns': ['../*', '../../*']
      }]
    }
  }
];
```

**Result of import ordering:**

```typescript
// 1. Node.js built-ins
import fs from 'fs';
import path from 'path';

// 2. External dependencies
import express from 'express';
import { z } from 'zod';

// 3. Internal aliases
import { config } from '@config/environment';
import { UserService } from '@services/user';
import type { User } from '@types/user';

// 4. Parent imports
import { helper } from '../utils';
import { constants } from '../../constants';

// 5. Sibling imports
import { utils } from './utils';
import type { LocalType } from './types';
```

### Path Alias Validation

Ensure path aliases resolve correctly:

```javascript
// eslint.config.js
export default [
  {
    settings: {
      'import/resolver': {
        typescript: {
          alwaysTryTypes: true,
          project: './tsconfig.json'
        },
        node: {
          paths: ['src'],
          extensions: ['.ts', '.tsx', '.js', '.jsx']
        }
      }
    },
    rules: {
      // Ensure imports can be resolved
      'import/no-unresolved': 'error',
      
      // Prevent cyclical dependencies
      'import/no-cycle': ['error', { maxDepth: 2 }]
    }
  }
];
```

---

## 37.4 Strict Linting Rules for Type Safety

Maximum strictness configuration for production applications.

### Strict Configuration

```javascript
// eslint.config.js
export default [
  {
    files: ['**/*.ts', '**/*.tsx'],
    rules: {
      // Ban types that are considered dangerous
      '@typescript-eslint/ban-types': ['error', {
        'types': {
          '{}': {
            'message': 'Use Record<string, unknown> instead',
            'fixWith': 'Record<string, unknown>'
          },
          'object': {
            'message': 'Use Record<string, unknown> instead'
          },
          'Function': {
            'message': 'Use specific function type instead'
          }
        }
      }],
      
      // Enforce explicit accessibility modifiers
      '@typescript-eslint/explicit-member-accessibility': ['error', {
        'accessibility': 'explicit',
        'overrides': {
          'constructors': 'no-public'
        }
      }],
      
      // Consistent naming conventions
      '@typescript-eslint/naming-convention': [
        'error',
        {
          'selector': 'default',
          'format': ['camelCase'],
          'leadingUnderscore': 'allow',
          'trailingUnderscore': 'allow'
        },
        {
          'selector': 'variable',
          'format': ['camelCase', 'UPPER_CASE', 'PascalCase'],
          'leadingUnderscore': 'allow'
        },
        {
          'selector': 'typeLike',
          'format': ['PascalCase']
        },
        {
          'selector': 'enumMember',
          'format': ['UPPER_CASE']
        },
        {
          'selector': 'parameter',
          'format': ['camelCase'],
          'leadingUnderscore': 'allow'
        }
      ],
      
      // Prefer nullish coalescing
      '@typescript-eslint/prefer-nullish-coalescing': 'error',
      
      // Prefer optional chaining
      '@typescript-eslint/prefer-optional-chain': 'error',
      
      // Require explicit boolean comparisons
      '@typescript-eslint/strict-boolean-expressions': ['error', {
        'allowNullableObject': true,
        'allowNullableBoolean': false,
        'allowNullableString': false,
        'allowNullableNumber': false,
        'allowAny': false
      }],
      
      // No console in production
      'no-console': ['warn', { 'allow': ['warn', 'error'] }],
      
      // Prefer const/let
      'prefer-const': 'error',
      'no-var': 'error',
      
      // No implicit returns
      '@typescript-eslint/no-implied-eval': 'error',
      
      // No non-null assertions (use with caution)
      '@typescript-eslint/no-non-null-assertion': 'error',
      
      // Consistent type assertions
      '@typescript-eslint/consistent-type-assertions': ['error', {
        'assertionStyle': 'as',
        'objectLiteralTypeAssertions': 'never'
      }],
      
      // No explicit any
      '@typescript-eslint/no-explicit-any': 'error',
      
      // No unused expressions
      '@typescript-eslint/no-unused-expressions': 'error',
      
      // Return types on functions
      '@typescript-eslint/explicit-function-return-type': 'error',
      
      // Module boundary types
      '@typescript-eslint/explicit-module-boundary-types': 'error'
    }
  }
];
```

---

## 37.5 Custom Lint Rules

Create project-specific rules for architectural constraints.

### Creating a Custom Rule

```javascript
// eslint-rules/no-direct-database-import.js
module.exports = {
  meta: {
    type: 'problem',
    docs: {
      description: 'Prevent direct database imports in controllers',
      category: 'Architecture',
      recommended: true
    },
    schema: []
  },
  create(context) {
    return {
      ImportDeclaration(node) {
        const importPath = node.source.value;
        const filename = context.getFilename();
        
        // Check if we're in a controller
        if (filename.includes('/controllers/')) {
          // Check if importing database directly
          if (importPath.includes('/database/') || importPath.includes('prisma')) {
            context.report({
              node,
              message: 'Controllers should not import database directly. Use services instead.'
            });
          }
        }
      }
    };
  }
};
```

**Using custom rules:**

```javascript
// eslint.config.js
import customRules from './eslint-rules/index.js';

export default [
  {
    plugins: {
      'custom': {
        rules: {
          'no-direct-database-import': customRules['no-direct-database-import']
        }
      }
    },
    rules: {
      'custom/no-direct-database-import': 'error'
    }
  }
];
```

---

## 37.6 Pre-commit Hooks

Automate linting and formatting before commits to ensure code quality.

### Husky and lint-staged Setup

```bash
npm install -D husky lint-staged
npx husky init
```

**.husky/pre-commit:**

```bash
#!/bin/sh
. "$(dirname "$0")/_/husky.sh"

npx lint-staged
```

**package.json:**

```json
{
  "lint-staged": {
    "*.{ts,tsx}": [
      "eslint --fix",
      "prettier --write"
    ],
    "*.{json,md}": [
      "prettier --write"
    ]
  }
}
```

**Alternative: Nano-staged (faster)**

```bash
npm install -D nano-staged
```

```json
// package.json
{
  "nano-staged": {
    "*.{ts,tsx}": [
      "eslint --fix",
      "prettier --write"
    ]
  }
}
```

### Commit Message Linting

```bash
npm install -D @commitlint/config-conventional @commitlint/cli
```

**commitlint.config.js:**

```javascript
export default {
  extends: ['@commitlint/config-conventional'],
  rules: {
    'type-enum': [2, 'always', [
      'feat', 'fix', 'docs', 'style', 'refactor', 
      'perf', 'test', 'chore', 'ci', 'build'
    ]],
    'scope-enum': [2, 'always', [
      'api', 'ui', 'db', 'auth', 'core', 'utils'
    ]],
    'subject-case': [2, 'always', 'sentence-case']
  }
};
```

**.husky/commit-msg:**

```bash
npx --no -- commitlint --edit ${1}
```

---

## 37.7 CI/CD Integration

Automate quality checks in continuous integration pipelines.

### GitHub Actions Workflow

```yaml
# .github/workflows/quality.yml
name: Code Quality

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main, develop]

jobs:
  lint-and-type-check:
    runs-on: ubuntu-latest
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: '20'
          cache: 'npm'
      
      - name: Install dependencies
        run: npm ci
      
      - name: Type check
        run: npm run type-check
      
      - name: Lint
        run: npm run lint
      
      - name: Check formatting
        run: npm run format:check
      
      - name: Run tests
        run: npm run test:ci
      
      - name: Upload coverage
        uses: codecov/codecov-action@v3
        with:
          files: ./coverage/lcov.info
```

### GitLab CI

```yaml
# .gitlab-ci.yml
stages:
  - quality

variables:
  NODE_VERSION: "20"

lint:
  stage: quality
  image: node:${NODE_VERSION}
  script:
    - npm ci
    - npm run lint
    - npm run format:check
  only:
    - merge_requests
    - main

type-check:
  stage: quality
  image: node:${NODE_VERSION}
  script:
    - npm ci
    - npm run type-check
  only:
    - merge_requests
    - main
```

---

## 37.8 TypeScript Compiler vs Linting

Understanding the distinction between tsc and ESLint helps optimize workflows.

### Key Differences

| Aspect | TypeScript Compiler (tsc) | ESLint |
|--------|--------------------------|--------|
| **Primary Purpose** | Type checking and compilation | Code style and pattern enforcement |
| **Speed** | Slower (full type analysis) | Faster (AST parsing only, unless type-aware) |
| **Output** | JavaScript files + declarations | No output, only diagnostics |
| **Scope** | Type correctness | Code quality, style, best practices |
| **Required** | Yes, for building | Optional but recommended |

### Optimizing Checks

```json
// package.json scripts for efficient workflows
{
  "scripts": {
    // Fast check for development (no emit)
    "type-check": "tsc --noEmit",
    
    // Watch mode for development
    "type-check:watch": "tsc --noEmit --watch",
    
    // Lint only changed files (faster)
    "lint:changed": "eslint . --ext .ts --cache",
    
    // Full lint with type checking (slower, for CI)
    "lint:full": "tsc --noEmit && eslint . --ext .ts",
    
    // Pre-commit (fastest)
    "precommit": "lint-staged"
  }
}
```

### Editor Integration

Configure VS Code for real-time feedback:

```json
// .vscode/settings.json
{
  "editor.defaultFormatter": "esbenp.prettier-vscode",
  "editor.formatOnSave": true,
  "editor.codeActionsOnSave": {
    "source.fixAll.eslint": "explicit",
    "source.organizeImports": "explicit"
  },
  "typescript.tsdk": "node_modules/typescript/lib",
  "typescript.enablePromptUseWorkspaceTsdk": true,
  "eslint.validate": [
    "javascript",
    "javascriptreact",
    "typescript",
    "typescriptreact"
  ],
  "eslint.workingDirectories": [
    { "mode": "auto" }
  ]
}
```

---

## 37.9 Chapter Summary and Exercises

### Chapter Summary

This chapter covered code quality tooling for TypeScript:

**Key Concepts:**

1. **ESLint + TypeScript**: `@typescript-eslint/parser` enables ESLint to understand TypeScript. Type-aware rules (`no-floating-promises`, `strict-boolean-expressions`) use the TypeScript compiler API for deeper analysis.

2. **Prettier Integration**: `eslint-config-prettier` disables conflicting ESLint rules. Run Prettier as an ESLint rule or separately. Prettier handles formatting; ESLint handles code quality.

3. **Import Organization**: `eslint-plugin-import` with path aliases ensures consistent import ordering, prevents circular dependencies, and validates that imports resolve correctly.

4. **Strict Rules**: `no-explicit-any`, `explicit-function-return-type`, and `strict-boolean-expressions` enforce type safety at the linting level, catching issues that might pass type checking but indicate code smell.

5. **Custom Rules**: Project-specific ESLint rules enforce architectural patterns (e.g., preventing direct database access from controllers) using the AST.

6. **Pre-commit Hooks**: `husky` + `lint-staged` run linters only on changed files before commits, ensuring quality without slowing down development.

7. **CI/CD**: Automated quality checks in pipelines prevent merging code that fails type checking, linting, or formatting.

8. **Compiler vs Linter**: TypeScript checks types; ESLint checks patterns. Use both—tsc for correctness, ESLint for style and best practices.

### Practical Exercises

**Exercise 1: Configure Strict ESLint**

Set up an ESLint configuration for a TypeScript project with:
- Type-aware rules enabled
- Import ordering with path aliases
- Prettier integration
- Custom naming conventions (interfaces prefixed with I, types with T)
- Ban on `console.log` (allow `console.error`)

**Exercise 2: Custom Architectural Rule**

Create an ESLint rule that prevents:
- Direct imports of the database client in API route handlers
- Imports from `../..` (grandparent directories)
- Relative imports when an alias is available

**Exercise 3: Pre-commit Pipeline**

Configure a pre-commit hook that:
- Runs type checking
- Lints only changed files
- Formats code
- Prevents commits to `main` branch
- Validates commit messages follow conventional commits format

**Exercise 4: CI Quality Gate**

Set up a GitHub Actions workflow that:
- Runs on Node.js 18, 20, and 22
- Caches dependencies
- Runs type check, lint, format check, and tests in parallel jobs
- Uploads coverage to Codecov
- Fails if coverage drops below 80%

**Exercise 5: Migration Strategy**

Given a JavaScript project migrating to TypeScript:
- Configure ESLint to allow `any` temporarily but warn
- Set up gradual strictness increase (strictNullChecks first, then strictFunctionTypes, etc.)
- Create a script to track TypeScript error count over time
- Configure VS Code to show type errors as warnings during migration

### Additional Resources

- **typescript-eslint**: https://typescript-eslint.io/
- **Prettier**: https://prettier.io/docs/en/index.html
- **ESLint Rules**: https://eslint.org/docs/rules/
- **Husky**: https://typicode.github.io/husky/
- **lint-staged**: https://github.com/okonet/lint-staged

---

## Coming Up Next: Chapter 38 - Debugging TypeScript

In the next chapter, we will explore debugging strategies for TypeScript applications:

- Source maps configuration and generation
- Debugging in VS Code (launch.json configurations)
- Chrome DevTools debugging for frontend
- Node.js debugger and inspect protocol
- Debugging tests with breakpoints
- Advanced debugging techniques (conditional breakpoints, logpoints)
- Remote debugging
- Docker debugging setup

Effective debugging skills are essential for diagnosing issues in type-safe code, understanding runtime behavior that differs from compile-time expectations, and navigating the compiled JavaScript while working with TypeScript source.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='36. testing_typescript_code.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='38. debugging_typescript.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
